# ETL Demo Notebook

This notebook demonstrates the ingestion of AdventureWorks (SQL Server) and Azure Open Datasets (weather/demographics), cleaning, and producing an analytics-ready CSV. Fill `.env` and ensure SQL Server is reachable.

**Run order:** 1) Configure `.env` 2) Start SQL Server (docker or local) 3) Place AdventureWorks DB or use existing DB 4) Run cells.

In [2]:
from dotenv import load_dotenv
load_dotenv()
print('Loaded env vars from .env')

Loaded env vars from .env


In [2]:
# Install dependencies (uncomment if needed)
!pip3.11 install -r requirements.txt

In [3]:
from src.connectors.sqlserver_adventureworks import get_engine_from_env, get_sales_transactions
engine = get_engine_from_env()
sales = get_sales_transactions(engine, start_date=None)
print('sales rows:', len(sales))
sales.head()

sales rows: 31465


,SalesOrderID,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,CustomerID,TerritoryID,TotalDue
0,75084,2014-06-30,2014-07-12,2014-07-07,5,True,SO75084,11078,6,132.6000
1,75085,2014-06-30,2014-07-12,2014-07-07,5,True,SO75085,11927,1,18.7187
2,75086,2014-06-30,2014-07-12,2014-07-07,5,True,SO75086,28789,4,8.7848
3,75087,2014-06-30,2014-07-12,2014-07-07,5,True,SO75087,11794,4,38.6640
4,75088,2014-06-30,2014-07-12,2014-07-07,5,True,SO75088,14680,9,125.9258


In [4]:
# Load Azure enrichment CSVs (if present)
import pandas as pd
from pathlib import Path
w = Path('data/raw/azure/weather_daily.csv')
if w.exists():
    weather = pd.read_csv(w)
    print('weather rows:', len(weather))
    display(weather.head())
else:
    print('No weather file found at', w)


weather rows: 120


,StationID,latitude,longitude,Year,Month,Day,DryBulbCelsius,WetBulbFarenheit,WetBulbCelsius,DewPointFarenheit,DewPointCelsius,RelativeHumidity,WindSpeed,WindDirection
0,10397,47.149757,-120.805769,2019,11,9,14.0,56,13.2,55,13.0,93,6,230
1,10397,47.149757,-120.805769,2019,11,10,12.2,47,8.4,40,4.4,59,7,320
2,10397,47.149757,-120.805769,2019,11,11,10.6,40,4.5,25,-3.9,36,8,340
3,10397,47.149757,-120.805769,2019,11,12,8.3,46,7.5,44,6.7,89,15,80
4,10397,47.149757,-120.805769,2019,11,13,4.4,39,3.9,38,3.3,93,8,50


In [ ]:
# Run the packaged ETL job

from src.etl.jobs.job_seed_and_ingest import run

if __name__ == "__main__":
    run()

Loading sales transactions...
Loading order details...
Loading customers...


/Users/amani/Desktop/CV Projects Data Science/Data_Automation/src/etl/jobs/job_seed_and_ingest.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.strip()


Merging header + details...
Weather columns: ['StationID', 'latitude', 'longitude', 'Year', 'Month', 'Day', 'DryBulbCelsius', 'WetBulbFarenheit', 'WetBulbCelsius', 'DewPointFarenheit', 'DewPointCelsius', 'RelativeHumidity', 'WindSpeed', 'WindDirection']
Wrote data/analytics/analytics_ready.csv


## Next steps

- Connect `data/analytics/analytics_ready.csv` to Power BI and design dashboards.
- Add data quality checks (Great Expectations) and scheduling (Airflow/Azure Functions).